# Import library

In [26]:
import requests
import json

# Helper function for REST call : GET / PUT / POST /DELETE

In [27]:
#helper
def get(url):
    """REST CALL : GET"""
    reponse = requests.get(url, verify=False)
    if (reponse.ok):
        val = reponse.json()
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def delete(url ):
    """REST CALL : DELETE"""
    reponse = requests.delete(url,verify=False)
    if (reponse.ok):
        val = "data deleted"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def put(url, json):
    """REST CALL : PUT"""
    reponse = requests.put(url, json=json, verify=False)
    if (reponse.ok):
        val = "build"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val

def post(url, json):
    """REST CALL : POST"""
    reponse = requests.post(url, json=json, verify=False,headers={'Connection':'close'})
    if(reponse.ok):
        if(reponse.status_code==201):
            val = reponse.headers._store['location'][1]
        else:
            val= reponse.json()
    elif(reponse.status_code==400):
        val= reponse.text
    else:
        status = reponse.raise_for_status()
        val =  "error : " + status
    reponse.close()
    return val

In [44]:
#use the API unitary price to price a FXO with the corresponding measures
#we use the input defined by FCP and use the FCP pricer
#since the API is not defined we use the generic pricer and we set by default the non used parameter
def unitary(unitary_price):
    url_price = url_fcp_swagger+"pricing/pricer/"
    unitary_price["resultHandlerId"] = "Report"
    unitary_price["resultHandlerConfigId"] = "id-Default"
    unitary_price["marketDataProviderId"] = "PE_STORE_MDP"
    unitary_price["referenceCurrency"] = "id/USD"    
    unitary_price["scenarioContexts"] = []
    unitary_price["scenarioContexts"].append({"outputCashflows" : unitray_price["outputCashflows"]})
    unitary_price["scenarioContexts"].append({"outputPastCashflows" : unitray_price["outputPastCashflows"]})
    unitary_price["scenarioContexts"].append({"resultsByLeg" : unitray_price["resultsByLeg"]})
    unitary_price["scenarioContexts"].append({"id" : "base"})
    unitary_price["scenarioContexts"].append({"measureGroupIds" : unitray_price["groupIds"]})
    unitary_price["perimeter"] = {"trade" : {"tradeId" : unitary_price["trade"]["tradeId"]["id"]}}
    return(post(url=url_price,json=unitary_price))
    

# URL LINK : SWAGGER FCC / FCP

In [34]:
url_fcp_swagger = "https://fr1pslcmf05:8770/api/"
## url to create currency static data
fcp_yc_definition = url_fcp_swagger+"pricing/Store/MarketData/YieldCurveDefinitions/"
fcp_yc_quote = url_fcp_swagger + "pricing/Store/MarketData/YieldCurves/"
## url to create currency static data
fcp_currency_static = url_fcp_swagger+"pricing/Store/StaticData/Currencies/"
fcp_CurrencyPair = url_fcp_swagger+"pricing/Store/StaticData/CurrencyPairs/"
#url to push the Fx rate to the store
fcp_fxPair = url_fcp_swagger+"pricing/Store/MarketData/FxRates"
#url for marketdata set
fcp_marketData_set = url_fcp_swagger+"pricing/marketDataSetIds"
#FXO trade
fcp_FXO_store = url_fcp_swagger+"Store/Trade/FxOption"
#pricing config 
fcp_config = url_fcp_swagger+"pricing/configs/"
#measure group
fcp_measure = url_fcp_swagger+"pricing/measureGroups/"

## Synthetic function for Fx static / Fx pair / Fx Yield curve

In [ ]:
def fx_static_pair(currency_dom, currency_for, forDom):
    post(url = fcp_currency_static, json=currency_dom)
    post(url = fcp_currency_static, json=currency_for)
    post(url = fcp_CurrencyPair, json=forDom)
        

# Create static data

In [ ]:
currency_dom = { 'id': '$DOM', 'isoCode': 'DOM'}
currency_for = {'id' : '$FOR', 'isoCode' : 'FOR'}
input_forDom =  {"currency1" : "FOR","currency2" : "DOM", "id":"$FOR/$DOM" , "name": "FOR/DOM", "spotLag" : 0}

In [ ]:
fx_static_pair(currency_dom, currency_for, input_forDom)
#check the static
print(get(url=fcp_currency_static+"$DOM"))
print(get(url=fcp_currency_static+"$FOR"))
print(get(url=fcp_CurrencyPair+'$FOR/$DOM'))

# Create FX Pair

In [ ]:
MD = get(url=fcp_marketData_set)
print(MD)

In [ ]:
fxPair = {
  "date": "2019-03-19",
  "marketSet": "DEFAULT",
  "name": "FOR/DOM",
  "value": 1.12
}
url_output = post(url = fcp_fxPair, json=fxPair)
#check the val
print(get(url=url_output))

## Create Yield Curve Definition

In [ ]:
yc_definition = {
    "name": "DOM_TEST",
    "currency": "DOM",
    "switchDates": [
        0,
        6
        ],
    "interpolationVariables": [
        "zeroCouponRate",
        "discountFactor",
        "zeroCouponRate"
        ],
    "interpolationMethods": [
        "flat",
        "linear",
        "flat"
        ],
    "zeroCouponFormula": "exponential",
    "zeroCouponBasis": "ACT/365.FIXED"
}

In [ ]:
#We push the static currency into FCP store
yc_def = post(url = fcp_yc_definition, json=yc_definition)
#Check the pair
print(get(url=yc_def))

## Create Yield curve

In [ ]:
MD = get(url=fcp_marketData_set)
print(MD)

In [ ]:
yc_DF = {
    "name": "DOM_TEST",
    "date": "2019-03-19",
    "marketSet": "DEFAULT",
    "valuesType": "discountFactor",
    "values": {
        "2020-02-08": 0.999800089973,
        "2020-03-12": 0.9956852186623,
        "2020-05-10": 0.9865951285774,
        "2020-08-10": 0.9729177720202,
        "2021-02-07": 0.9443724335959,
        "2025-02-09": 0.8493570595195,
        "2028-02-07": 0.7754530001423
        }
    }

In [ ]:
yc = post(url = fcp_yc_quote,json=yc_DF)
print(get(url=yc))

# price a trade defined in the store

## Step 1 : Define pricingConfig

### Step a : define risk factor resolver

### Step b : define finite difference scheme config

### Step c : define the pricing config

In [25]:
#TO BE CONFIRMED : 
#pricer Resolver : only GET how to define it
#risk factor resolver : can define it via a POST
#finitedifferenceSchemesConfig : can define it via a POST
pricing_config_id = "e26f34d7-8192-4264-b405-33b6b79afedb"
pricing_config = get(fcp_config+"e26f34d7-8192-4264-b405-33b6b79afedb")

## Step 2 : Get  marketDataSet

In [30]:
MD = get(url=fcp_marketData_set)
print(MD)

[{'id': 'id-Default', 'name': 'DEFAULT'}]


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## Step 3 : define the measure groups

In [35]:
measure_group = {
  "measureIds": [
    "NPV","FXD_Delta_Spot","FXD_Gamma","FXD_Rho_Foreign","FXD_Rho_Domestic","FXD_Theta","FXD_Vega"
  ],
  "name": "FXO_OTP"
}

In [39]:
###To be discussed : maybe it is better to pass the list than create a measure group
measure = post(url=fcp_measure,json=measure_group)
print(get(fcp_measure+measure['id']))

C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': 'aa41a417-4327-476b-9868-fbfb4dead8c5', 'name': 'FXO_OTP', 'measureIds': ['FXD_Delta_Spot', 'FXD_Gamma', 'FXD_Rho_Domestic', 'FXD_Rho_Foreign', 'FXD_Theta', 'FXD_Vega', 'NPV']}


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## Step 4 : get the trade ID

In [ ]:
fxo_trade = "FXOptionVanilla_28"

## Step 5 : price

In [45]:
#To be confirmed : do we need of :
#outputCashflows, outputPastCashflows, resultsByLeg, 
#forceFiniteDifference : always true or false for vanilla)
#version , draft not yet used
#should we precise the reference currency here?

unitaryPrice ={
  "asOfDate": "2016-07-04",
  "pricingConfigId": "e26f34d7-8192-4264-b405-33b6b79afedb",
  "marketDataSetId": "id-Default",
  "pricingMethod": "THEORETICAL",
  "measures": {
    "groupIds": [
      "aa41a417-4327-476b-9868-fbfb4dead8c5"
    ]
  },
  "outputCashflows": True,
  "outputPastCashflows": True,
  "resultsByLeg": True,
  "forceFiniteDifferences": False,
  "trade": {
    "tradeId": {
      "id": "FXOptionVanilla_28",
      "version": "string",
      "draft": "string"
    }
  }
}

In [46]:
result = unitaryPrice(unitaryPrice)

TypeError: 'dict' object is not callable

## Clean MarketData

In [20]:
#clean static and FX pair
print(delete(url=fcp_CurrencyPair+'$FOR/$DOM'))
print(delete(url=fcp_currency_static+"$DOM"))
print(delete(url=fcp_currency_static+"$FOR"))
print(delete(url=url_output))
#clean yield curve
print(delete(url=yc_def))
print(delete(url=yc))
#clean measure group
print(delete(fcp_measure+measure['id']))

C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data deleted


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data deleted


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data deleted
data deleted

C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data deleted
data deleted


C:\Users\jerpetit\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
